In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/twitch_data

/content/drive/MyDrive/twitch_data


In [5]:
%ls

1013713136.mp4                  raid_VNCEOOfficial.html  sentiments3
chat_df.csv                     sentiments               sentiments4
chat_features_womeaningful.csv  sentiments1              sentiments5
data_vid_df_l.csv               sentiments2              video_df_merge.csv


In [6]:
chat_df = pd.read_csv('chat_df.csv',lineterminator='\n') 
chat_df.columns

Index(['Unnamed: 0', 'video_url', 'timestamp', 'time_in_seconds', 'author_id',
       'author_name', 'author_bio', 'messages'],
      dtype='object')

In [7]:
#chat_df['video_id'] = chat_df['video_url'].str.strip().str[-10:]

In [8]:
video_df = pd.read_csv('video_df_merge.csv',index_col= 0)
print(video_df.columns)

video_df.drop_duplicates(subset=['id'], keep='first',inplace=True)

Index(['id', 'stream_id', 'user_id', 'user_login', 'user_name', 'title',
       'description', 'created_at', 'published_at', 'url', 'thumbnail_url',
       'viewable', 'view_count', 'language', 'type', 'duration',
       'muted_segments', 'duration_min', 'broadcaster_type',
       'broadcast_description', 'broadcaster_profile_image_url',
       'broadcaster_offline_image_url', 'broadcaster_created_at'],
      dtype='object')


## Basic chat features

#### num_messages, message_len, messages_per_min   
message_per_minute is a crude measure of 'roar of the corwd', when a bunch of messages appear like a waterfall. Because if there's no message in this minute, this minute is ignored when calculating this variable. 

In [9]:
# basic chat features
chat_features_df = chat_df.groupby('video_url').agg({'messages':'count', # number of messages
                                                     }).reset_index()
chat_features_df.rename(columns={'messages':'num_messages'},inplace=True)
chat_features_df['video_id'] = chat_features_df['video_url'].str.strip().str[-10:]

In [10]:
## basic chat feature: number of avg_messages_per_minute
# merge in some video_level variables
variables = ['id','user_id','user_name','duration_min','created_at','title','view_count','broadcaster_type','broadcast_description','broadcaster_created_at']
chat_features_df = pd.merge(chat_features_df, video_df[variables], how="left", left_on='video_id',right_on = 'id')
chat_features_df.drop('id', axis=1, inplace=True)  

In [11]:
print(chat_features_df['video_url'].nunique())
print(chat_features_df['user_id'].nunique())

91
12


In [12]:
# video_df.user_name.unique()
chat_features_df.user_name.unique()

array(['VNCEOfficial', 'AkzOnTop', 'PatProzyy', 'prxd4v41', 'Methodz',
       'General_Mars', 'Rainbow6', 'SypherPK', 'iiTzTimmy',
       'moistcr1tikal', 'Murda10k', 'zackrawrr'], dtype=object)

In [13]:
chat_features_df['video_url'][25:30]

25    https://www.twitch.tv/videos/1604098127
26    https://www.twitch.tv/videos/1604165847
27    https://www.twitch.tv/videos/1604458780
28    https://www.twitch.tv/videos/1604733184
29    https://www.twitch.tv/videos/1604878671
Name: video_url, dtype: object

In [14]:
temp_table = chat_df[chat_df['messages'].str.contains('raid',na=False)]

In [ ]:
temp_table

## Scrape raid data


In [17]:
import requests # to fetch the raw HTML from our page
import csv
from bs4 import BeautifulSoup # to extract just the information we are looking for

In [77]:
def get_data(soup_object):
    table = soup.find('table', class_="w-full rounded-lg bg-card")

    # scrapes the table head element
    outputHead = []
    for th in table.find_all('th'):  # find_all() function to find the elements we are looking for
        outputHead.append(th.text)  #write to a output list

    # traverses all the table rows
    outputRows = []
    for tr in table.find_all('tr'):
        tableColumns = tr.find_all('td')
        outputRow = []
        for tc in tableColumns:
            outputRow.append(tc.text.replace('\n', ''))
        outputRows.append(outputRow)

    return outputHead, outputRows

In [85]:
with open("raid_VNCEOOfficial.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')

outputHead, outputRows = get_data(soup)
raid_df= pd.DataFrame(outputRows,columns=outputHead)[1:].set_index(outputHead[0])

raid_df.head()

In [93]:
raid_file_list = ['raid_VNCEOOfficial.html', 'raid_patprozyy.html', 'raid_prxd4v41.html', 'raid_Methodz.html',
        'raid_Rainbow6.html', 'raid_SypherPK.html', 'raid_iiTzTimmy.html',
       'raid_moistcr1tikal.html', 'raid_zackrawrr.html']

for i in range(len(raid_file_list)):
  with open('raid/' + raid_file_list[i] ) as fp:
    soup = BeautifulSoup(fp, 'html.parser')
  outputHead, outputRows = get_data(soup)

  if i == 0:
    raid_df= pd.DataFrame(outputRows,columns=outputHead)[1:].set_index(outputHead[0])
  else:
    temp = pd.DataFrame(outputRows,columns=outputHead)[1:].set_index(outputHead[0])
    raid_df.append(temp)

In [94]:
# This saves to google drive directly
raid_df.to_csv('raid_df.csv')

In [96]:
raid_df.head()
print(raid_df.shape)

(230, 6)


Now we're gonna pre-process these raid data.

## Ignore code down below

In [ ]:
# makes a GET request to a URL and returns the text from that page.
def get_page(url):
    return requests.get(url).content

In [ ]:
page = get_page('https://streamscharts.com/tools/raid-finder/vnceofficial')
page

In [ ]:
soup = BeautifulSoup(page, 'html.parser') # parses the document using a HTML parser

In [34]:
lists = soup.find_all('th')

In [ ]:
lists

In [ ]:
import re

In [30]:
url = "raid_VNCEOOfficial.html"
page = open(url)

In [32]:
page.text()

AttributeError: ignored

In [19]:
soup = BeautifulSoup(page,'html.parser')

In [ ]:
print(table)

In [ ]:
header = table.find_all('th')

AttributeError: ignored

In [ ]:
print(table)

None


In [ ]:
soup